# 🥉 Camada Bronze — Ingestão Open Brewery com Auto Loader

Este notebook implementa a camada **Bronze** da arquitetura **Medallion** no Databricks utilizando:

- Auto Loader (cloudFiles)
- Unity Catalog Volumes
- Delta Lake
- Streaming incremental com checkpoint

O objetivo da Bronze é armazenar os dados **exatamente como chegam da origem**, sem qualquer transformação.


## 📥 Origem dos dados

O Databricks Community Edition não permite chamadas externas para APIs.

Por isso, os dados da Open Brewery API são coletados externamente e enviados para um Volume do Unity Catalog: `/Volumes/bees-teste-jp/default/landing/openbrewery/`

Código usado para coletar do dados:

```python
import requests
import json

all_data = []
page = 1

while True:
    url = f"https://api.openbrewerydb.org/v1/breweries?page={page}&per_page=200"
    resp = requests.get(url)
    data = resp.json()
    
    if not data:
        break
        
    all_data.extend(data)
    page += 1

print(len(all_data))

with open("breweries_raw.json", "w") as f:
    json.dump(all_data, f)

from google.colab import files
files.download("breweries_raw.json")`
```



Cada execução da API gera um novo arquivo JSON nessa pasta.

Isso simula um cenário real de Data Lake, onde arquivos chegam em uma área de *landing*.



## 🚀 Por que usar Auto Loader?

Em vez de usar `spark.read.json`, utilizamos o **Auto Loader**:

`spark.readStream.format("cloudFiles")`

Vantagens:

- Processa apenas arquivos novos (incremental)
- Mantém estado via checkpoint
- Suporta evolução de schema
- Escalável para grandes volumes de arquivos

## 📖 Leitura de JSON com `multiLine=true`

O JSON da Open Brewery vem no formato de **array único**:

[
  {...},
  {...}
]

O Spark por padrão espera **um JSON por linha**.

Por isso utilizamos:

`.option("multiLine", "true")`




## 🏗️ Código de leitura da Bronze


In [0]:
df_bronze = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("multiLine", "true")
    .option("cloudFiles.schemaLocation", "/Volumes/bees-teste-jp/default/checkpoints/openbrewery_schema")
    .load("/Volumes/bees-teste-jp/default/landing/openbrewery/")
)


## 💾 Escrita na tabela Delta Bronze

Os dados são escritos em uma tabela Delta utilizando streaming.

O `checkpointLocation` é o que garante que:

- O Auto Loader saiba quais arquivos já foram ingeridos
- O streaming possa continuar de onde parou

## ⏱️ Uso do `trigger(availableNow=True)`

Esse modo executa o streaming como um **batch inteligente**:

- Processa todos os arquivos novos disponíveis
- Finaliza a execução
- Mantém o estado salvo no checkpoint

Na próxima execução, apenas novos arquivos serão lidos.


In [0]:
(df_bronze.writeStream
    .format("delta")
    .option("checkpointLocation", "/Volumes/bees-teste-jp/default/checkpoints/openbrewery_checkpoint")
    .trigger(availableNow=True)
    .toTable("`bees-teste-jp`.default.bronze_openbrewery")
)


In [0]:
%sql
SELECT COUNT(*) FROM `bees-teste-jp`.default.bronze_openbrewery